In [ ]:
from helper import * 
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

path = 'data'
df = load_data(f'{path}/driving_log.csv')
df = balance_data(df, 'Steering')
df

The car go straight most of the time that makes steering values around 0 extremely higher than the other. Drop the steering data to balance the whole dataset. 

In [ ]:
img_path, steering = load_data_to_array(path, df)
# fixed value for random_state to keep train and test datasets same each time
X_train, X_val, Y_train, Y_val = train_test_split(img_path, steering, test_size=0.2, random_state=1) 
X_train, Y_train = img_preprocess_pipeline(X_train, Y_train, 1)
X_val, Y_val = img_preprocess_pipeline(X_val, Y_val, 0 )

print(X_train.shape)
print(Y_train.shape)
print("Training imgs: ", len(X_train))
print("Test imgs: ", len(X_val))

In [ ]:
import wandb
from wandb.keras import WandbCallback
from wandb_config import sweep_config

wandb.init(project='cnn_thesis')
sweep_config = sweep_config

sweep_id = wandb.sweep(sweep_config, project='cnn_thesis')
log_callback = WandbCallback(log_evaluation=True)

In [ ]:

def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        optim = build_optimizer(config.optimizer, config.learning_rate)
        model = build_network(activation=config.activation, optimizer=optim)
        
        model.fit(X_train, Y_train, validation_data=(X_val, Y_val), batch_size=config.batch_size,
                        verbose=1, epochs=config.epochs, callbacks=[log_callback])

In [5]:
wandb.agent(sweep_id, train, count=20)

200/200 [==============================] - ETA: 0s - loss: 69267024.0000 - accuracy: 0.0815

INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best)... Done. 0.2s


200/200 [==============================] - 29s 141ms/step - loss: 69267024.0000 - accuracy: 0.0815 - val_loss: 1.8760 - val_accuracy: 0.0554
Epoch 2/49
200/200 [==============================] - ETA: 0s - loss: 2.0256 - accuracy: 0.0685

INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets


...

INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best)... Done. 0.2s


200/200 [==============================] - 25s 128ms/step - loss: 2.0120 - accuracy: 0.0685 - val_loss: 1.8563 - val_accuracy: 0.0554
Epoch 5/49
200/200 [==============================] - ETA: 0s - loss: 2.0028 - accuracy: 0.0685

INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets


INFO:tensorflow:Assets written to: c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best\assets
wandb: Adding directory to artifact (c:\Users\viet tran\Desktop\Masterthesis\Program\wandb\run-20221025_203420-33mwiwu2\files\model-best)... Done. 0.1s


200/200 [==============================] - 18s 91ms/step - loss: 2.0028 - accuracy: 0.0685 - val_loss: 1.8465 - val_accuracy: 0.0554
Epoch 6/49
200/200 [==============================] - ETA: 0s - loss: 1.9918 - accuracy: 0.0685

In [ ]:
''' 
Iterations: number of batches to complete one epoch 
Batches: number of training samples in one iteration 
Steps per epoch = number of training sample // batch size
'''
